In [3]:
import findspark
findspark.init()

In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder \
        .master("local") \
        .appName("WordCount") \
        .getOrCreate()

sc=spark.sparkContext

In [20]:
df=spark.read.text("file:///C:/Users/venka/PycharmProjects/pythonProject/dataset/about_us.txt")

#from pyspark.sql import explode(to convert list to records), split, col
df_count = (
    df.withColumn('word', explode(split(col('value'), ' ')))
       .groupBy('word')
       .count()
       .sort('count', ascending=False)
)

df_count.show()

+-------+-----+
|   word|count|
+-------+-----+
|  Spark|   18|
|    and|   17|
|    for|   15|
|    the|    9|
|     to|    8|
|     is|    7|
|     on|    7|
|     of|    7|
| Python|    7|
|       |    7|
|     in|    6|
|  Scala|    6|
|      a|    6|
|   Java|    5|
|version|    5|
| Apache|    4|
|   This|    4|
|  using|    3|
|    can|    3|
|  Arrow|    3|
+-------+-----+
only showing top 20 rows



In [22]:
spark

In [36]:
in_text = sc.textFile("file:///C:/Users/venka/PycharmProjects/pythonProject/dataset/about_us.txt")

in_text_flat = in_text.flatMap(lambda x: x.split(' ')) \
                    .map(lambda x: (x,1)) \
                    .reduceByKey(lambda x,y : x+y)

in_text_flat.collect()

[('This', 4),
 ('Tutorial', 1),
 ('is', 7),
 ('exclusive', 1),
 ('for', 15),
 ('the', 9),
 ('people', 1),
 ('who', 1),
 ('are', 3),
 ('interested', 1),
 ('in', 6),
 ('making', 1),
 ('their', 2),
 ('foot', 1),
 ('prints', 1),
 ('one', 2),
 ('of', 7),
 ('top', 1),
 ('niche,', 1),
 ('trending', 1),
 ('technology', 1),
 ('IT', 1),
 ('world.', 1),
 ('The', 1),
 ('content', 1),
 ('posted', 1),
 ('here', 1),
 ('suitable', 1),
 ('all', 2),
 ('learning', 1),
 ('professionals,', 1),
 ('Students,', 1),
 ('Teacher,', 1),
 ('', 7),
 ('Freshers', 1),
 ('etc.', 1),
 ('Here,', 1),
 ('we', 1),
 ('talk', 1),
 ('about', 1),
 ('BigData', 1),
 ('analytics,', 1),
 ('Azure', 2),
 ('Data', 2),
 ('Engineering,', 1),
 ('Spark', 18),
 ('Scala,', 2),
 ('with', 2),
 ('Python', 7),
 ('(PySpark),', 1),
 ('Functions,', 1),
 ('Machine', 1),
 ('Learning', 1),
 ('concepts', 1),
 ('and', 17),
 ('techniques,', 1),
 ('Natural', 1),
 ('Language', 1),
 ('Processing,', 1),
 ('MS-PowerApps,', 1),
 ('Visualization', 1),
 ('tool

In [37]:
from pyspark.sql.types import FloatType, ArrayType, StringType

@udf(ArrayType(ArrayType(StringType())))
def count_words(a:list):
    word_set = set(a)
    freq = [] #create frequency dict
    for word in word_set:
        freq.append([word,a.count(word)])
        
    return list(freq)

In [50]:
df_count_udf = (
    df.withColumn('wordCount',explode(count_words(split(col('Value'), ' ')))) \
            .withColumn("word",col("wordCount")[0]) \
            .withColumn("count",col("wordCount")[1]) \
            .drop('value', 'wordCount') \
            .sort("count", ascending = False)
)

df_count_udf.show()

+------+-----+
|  word|count|
+------+-----+
|   for|    6|
| Spark|    6|
|   and|    5|
|    on|    5|
| Scala|    5|
|   and|    4|
| Spark|    4|
|    in|    3|
| Spark|    3|
|   for|    3|
|   can|    3|
|    to|    3|
|   and|    3|
|   for|    3|
|Python|    3|
|Apache|    3|
| Arrow|    3|
|engine|    2|
|   the|    2|
|   SQL|    2|
+------+-----+
only showing top 20 rows

